In [ ]:
#!pip install -U torchtext==0.8.0

from torchtext.data import field,BucketIterator, TabularDataset
!pip install torchtext --upgrade

  Using cached torchtext-0.10.0-cp37-cp37m-manylinux1_x86_64.whl (7.6 MB)
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.8.0
    Uninstalling torchtext-0.8.0:
      Successfully uninstalled torchtext-0.8.0


In [ ]:


import csv 
import os 

import torch 
import torchtext
import torch.nn.functional as F

from torchtext.data import get_tokenizer
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import ngrams_iterator
from torchtext.data.metrics import bleu_score

import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import itertools
import string

import pandas as pd
import numpy as np
import multiprocessing
import os, io, json
import random

import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_1 = "/content/drive/MyDrive/Dataset/Dakshina/hi/hi.romanized.rejoined.dev.native.txt"
path_2 = "/content/drive/MyDrive/Dataset/Dakshina/hi/hi.romanized.rejoined.dev.roman.txt"
path_3 = "/content/drive/MyDrive/Dataset/Dakshina/hi/hindi_joined.csv"
pth_w2v_1 = "/content/drive/MyDrive/Dataset/Dakshina/hi/wordembed/w2v_native_1.model"
pth_w2v_r_2 = "/content/drive/MyDrive/Dataset/Dakshina/hi/wordembed/w2v_roman_1.model"

pth_ft_1 = "/content/drive/MyDrive/Dataset/Dakshina/hi/wordembed/fastText_native_1.model"
path_embedw2v_150_native = "/content/drive/MyDrive/Dataset/Dakshina/hi/wordembed/native_w2v_embed.pickle"
path_tokenizer_native = "/content/drive/MyDrive/Dataset/Dakshina/hi/tokenizer+/native_train_tokenizer.json"
path_xtrain_native = "/content/drive/MyDrive/Dataset/Dakshina/hi/xtrain_hi.pickle"

path_embedw2v_150_roman = "/content/drive/MyDrive/Dataset/Dakshina/hi/wordembed/roman_w2v_embed.pickle"
path_tokenizer_roman = "/content/drive/MyDrive/Dataset/Dakshina/hi/tokenizer+/roman_train_tokenizer.json"
path_ytrain_roman = "/content/drive/MyDrive/Dataset/Dakshina/hi/ytrain_hi.pickle"

In [ ]:
# for reproducibility
# refer https://pytorch.org/docs/stable/notes/randomness.html
SEED = 42


np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
native = []
with open(path_1,'r')as fp:
   lines= fp.read().split("\n")
   for line in lines:
     native.append(line)
native

In [ ]:
roman = []
with open(path_2,'r')as fp:
   lines= fp.read().split("\n")
   for line in lines:
     roman.append(line)
roman

In [ ]:
df1 = pd.DataFrame(native,columns=["Hindi"])
df2 = pd.DataFrame(roman)
df1["roman"] = df2


In [ ]:
count_max = 0 

lines = 0
for i,line in enumerate(native):
  count= 0
  line = line.split(" ")
  if len(line)<50:
    lines+=1
print(count_max,lines) 

0 4880


In [ ]:
print(len(native),len(roman))

5001 5001


In [ ]:
df1.to_csv(path_3)

In [ ]:
df = pd.read_csv(path_3,index_col=[0])
df.dropna(inplace=True,axis=0)

In [ ]:
df2 = df.copy()
for i,line in enumerate(df2["Hindi"]):
  line = line.split(" ")
  if len(line) > 50:
    df2.drop(index=i,axis=0,inplace = True)
len(df2)

4888

In [ ]:
df2.to_csv(path_3)

In [1]:
df2 = pd.read_csv(path_3,index_col=[0])

NameError: ignored

In [ ]:
hindi = df2["Hindi"]
roman = df2["roman"]

In [ ]:
from sklearn.model_selection import train_test_split

train , val = train_test_split(df2[["Hindi","roman"]], test_size=0.1)
train.to_csv("train.csv",index=None)
val.to_csv("val.csv",index=None)

In [ ]:
Field = field.Field

In [ ]:
tokenizer = get_tokenizer(None) 


def tokenize_(sentence):
  return [tokenizer(words) for words in sentence]

hi_text = Field(tokenize=tokenizer,use_vocab=True)
en_text = Field(tokenize=tokenizer,use_vocab=True,eos_token="<eos>",init_token="<sos>",is_target=True) 


['usne',
 'Vindhayvasini',
 'Devi',
 'ki',
 'puja',
 'ke',
 'liye',
 'prerit',
 'kiya.']

In [ ]:
data_fields = [('Hindi', hi_text), ('roman', en_text)]
train= TabularDataset(path = "/content/train.csv",
                            format='csv', skip_header=True,fields=data_fields)
val= TabularDataset(path = "/content/val.csv",
                            format='csv', skip_header=True,fields=data_fields)

hi_text.build_vocab(train.Hindi,val.Hindi)
en_text.build_vocab(train.roman,val.roman)

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cpu'

In [ ]:
train_dataloader,val_dataloader = torchtext.data.BucketIterator.splits(
    
                              # Datasets for iterator to draw data from
                              (train,val),

                              # Tuple of train and validation batch sizes.
                              batch_sizes=(32,32),

                              # Device to load batches on.
                              device=device, 

                              # Function to use for sorting examples.
                              sort_key=lambda x: len(x.Hindi),


                              # Repeat the iterator for multiple epochs.
                              repeat=False, 

                              # Sort all examples in data using `sort_key`.
                              sort=False, 

                              # Shuffle data on each epoch run.
                              shuffle=True,

                              # Use `sort_key` to sort examples in each batch.
                              sort_within_batch = False,
                              
                              )

In [ ]:
class Encoder(nn.Module):
    """ Encoder encodes the input sequence into a dense embedded context.
    """
    
    def __init__(self, input_size, embed_size, hidden_size, n_layers=1, dropout_rate=0.5):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout_rate = dropout_rate
        self.embedding = nn.Embedding(input_size, embed_size)
        self.rnn = nn.GRU(embed_size, hidden_size, n_layers, bidirectional=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
    

    def load_pretrained_embeddings(self, embeddings, fine_tune=False):
        self.embedding.weight = nn.Parameter(embeddings)
        for p in self.embedding.parameters():
            p.requires_grad = fine_tune
    
    
    def forward(self, x, lengths):
        """ x: (seq_len, batch_size)
            lengths: (batch_size)
        """
        lengths, perm_idx = lengths.sort(0, descending=True)
        
        x = x.T[perm_idx]
        x = x.T

        embed = self.dropout(self.embedding(x))  # (seq_len, batch_size, embed_size)
        
        # packing the embedding sequence to avoid unnecessary computations
        # refer https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch
        packed_embed = pack_padded_sequence(embed, lengths.cpu().numpy())
        packed_outputs, hidden = self.rnn(packed_embed)
        outputs, _ = pad_packed_sequence(packed_outputs)
        
        # outputs is a non-packed sequence of all hidden states from last layer
        # outputs: (seq_len, batch_size, hidden_size * 2)
        # hidden is the final non-padded time stepped state
        # hidden: (n_layers * 2, batch_size, hidden_size)
        
        # initial decoder hidden state
        hidden = F.relu(self.fc(torch.cat([h for h in hidden[-2:]], dim=1)))  # (batch_size, hidden_size)
        print(output.shape)
        print(hidden.shape)
        # outputs: (seq_len, batch_size, hidden_size * 2)
        # hidden: (batch_size, hidden_size)
        return outputs, hidden



In [ ]:
class BahdanauAttention(nn.Module):
    
    
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.hidden_size = hidden_size
        
        self.W1 = nn.Linear(hidden_size * 2, hidden_size, bias=False)
        self.W2 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.V = nn.Linear(hidden_size, 1, bias=False)
    
    
    def forward(self, hidden, enc_outputs, enc_masks):
        """ hidden: (batch_size, hidden_size)
            enc_outputs: (seq_len, batch_size, hidden_size * 2)
            enc_masks: (batch_size, seq_len)
        """
        seq_len = enc_outputs.shape[0]
        
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)  # (batch_size, seq_len, hidden_size)
        enc_outputs = enc_outputs.permute(1, 0, 2)  # (batch_size, seq_len, hidden_size * 2)
        
        # calculate alignment scores
        energy = F.relu(self.W1(enc_outputs) + self.W2(hidden))  # (batch_size, seq_len, hidden_size)
        scores = self.V(energy)  # (batch_size, seq_len, 1)
        scores = scores.squeeze(2)  # (batch_size, seq_len)
        
        # mask out invalid positions
        scores = scores.masked_fill(enc_masks == 0, float('-inf'))  # (batch_size, seq_len)
        
        # calculate the attention weights (prob) from alignment scores
        alphas = F.softmax(scores, dim=-1)  # (batch_size, seq_len)
        
        # calculate context vector
        context_vector = torch.bmm(alphas.unsqueeze(1), enc_outputs)  # (batch_size, 1, hidden_size * 2)
        
        # context_vector: (batch_size, 1, hidden_size * 2)
        # alphas: (batch_size, seq_len)
        return context_vector, alphas

In [ ]:
class DecoderWithAttention(nn.Module):
    """ decode the encoded dense embedding context into a sequence 
        with use of attention mechanism.
    """
    def __init__(self, output_size, embed_size, hidden_size, n_layers=1, dropout_rate=0.5):
        super(DecoderWithAttention, self).__init__()
        self.output_size = output_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout_rate = dropout_rate
        
        self.embedding = nn.Embedding(output_size, embed_size)
        self.rnn = nn.GRU(embed_size + hidden_size * 2, hidden_size, n_layers, bidirectional=False, dropout=0.6)
        self.attention = BahdanauAttention(hidden_size)
        self.fc1 = nn.Linear(embed_size + hidden_size + hidden_size * 2, output_size) #output_size
        
        self.dropout = nn.Dropout(0.4)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

        
        
    
    def load_pretrained_embeddings(self, embeddings, fine_tune=False):
        self.embedding.weight = nn.Parameter(embeddings)
        for p in self.embedding.parameters():
            p.requires_grad = fine_tune
    
    
    def forward(self, x, hidden, enc_outputs, enc_masks):
        """ x: (batch_size)
            hidden: (batch_size, hidden_size)
            enc_outputs: (seq_len, batch_size, hidden_size * 2)
            enc_masks: (batch_size, seq_len)
        """
        embed = self.dropout(self.embedding(x.unsqueeze(0)))  # (1, batch_size, embed_size)
        
        # calculate attention weights and context vector
        context_vector, alphas = self.attention(hidden, enc_outputs, enc_masks)
        
        # context_vector: (batch_size, 1, hidden_size * 2)
        # alphas: (batch_size, seq_len)
        
        context_vector = context_vector.permute(1, 0, 2)  # (1, batch_size, hidden_size * 2)
        
        # input to the rnn is a concatenation of embedding and weighted encoder states (context)
        rnn_input = torch.cat((embed, context_vector), dim=2)  # (1, batch_size, embed_size + hidden_size * 2)
        
        # update the rnn hidden state
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        # output: (seq_len=1, batch_size, hidden_size)
        # hidden: (n_layers=1, batch_size, hidden_size)
        # seq_len and n_layers are always equal to 1 for this decoder
        
        assert (output == hidden).all()
        
        embed = embed.squeeze(0)  # (batch_size, embed_size)
        output = output.squeeze(0)  # (batch_size, hidden_size)
        context_vector = context_vector.squeeze(0)  # (batch_size, hidden_size * 2)
        
        prediction = self.dropout(self.fc1(torch.cat((embed, output, context_vector), dim=1)))  # (batch_size, output_size)
        
        
        hidden = hidden.squeeze(0)  # (batch_size, hidden_size)
        
        # prediction: (batch_size, output_size)
        # hidden: (batch_size, hidden_size)
        # alphas: (batch_size, seq_len)
        return prediction, hidden, alphas


In [ ]:
import random
class Seq2SeqWithAttention(nn.Module):
    """ Seq2Seq wrapper for encoder and decoder.
    """
    
    def __init__(self, encoder, decoder, pad_idx):
        super(Seq2SeqWithAttention, self).__init__()
        self.pad_idx = pad_idx
        self.device = ("cuda" if torch.cuda.is_available() else 'cpu')
        self.encoder = encoder
        self.decoder = decoder
    
    
    def forward(self, source, target, teacher_force_ratio=0.5):
        """ source: (source_seq_len, batch_size)
            source_lengths: (batch_size)
            target: (target_seq_len, batch_size)
        """
        source_lengths =torch.LongTensor(list(map(self.length, source.T)))
        
        source_lengths = source_lengths.to(source.device)
        target_seq_len, batch_size = target.shape
        target_vocab_size = self.decoder.output_size
        
        # initialize tensor to store decoder outputs
        outputs = torch.zeros((target_seq_len, batch_size, target_vocab_size)).to(source.device)
        
        # outputs: (target_seq_len, batch_size, output_size)
        
        # encode the source sequence
        enc_outputs, hidden = self.encoder(source,source_lengths)
        
        # enc_outputs: (seq_len, batch_size, hidden_size * 2)
        # hidden: (batch_size, hidden_size)
        
        # create mask for source sequence
        enc_masks = (source != self.pad_idx).permute(1, 0)  # (batch_size, seq_len)
        
        # initial input to the decoder is the SOS_TOKEN
        x = target[0, :]  # (batch_size)
        
        for t in range(1, target_seq_len):
            # decode the encoded sequence for current time step
            output, hidden, _ = self.decoder(x, hidden, enc_outputs, enc_masks)
            
            # output: (batch_size, output_size)
            # hidden: (batch_size, hidden_size)
            
            # store the decoder outputs for current time step
            outputs[t] = output
            
            # get the highest predicted token (best word) for current time step
            top1 = output.argmax(dim=1)
           
            x = target[t] if random.random() < teacher_force_ratio else top1
        
        # outputs: (target_seq_len, batch_size, output_size)
        return outputs
    def length(self,vector):
        new_vec = []
        for vec in vector:
          if not vec==1:
              new_vec.append(vec)
        return len(new_vec)
    

In [ ]:
en_text.vocab.itos[3]

'<eos>'

In [ ]:
INPUT_SIZE = len(hi_text.vocab)  # source vocab size
OUTPUT_SIZE = len(en_text.vocab)  # target vocab size
ENC_EMBED_SIZE = 512
DEC_EMBED_SIZE = 256
HIDDEN_SIZE = 512
N_LAYERS = 1
DROPOUT_RATE = 0.5
N_EPOCHS = 50
CLIP = 0.5
PAD_IDX = 1
UNK_IDX = 0
SOS_IDX = 2
EOS_IDX = 3

In [ ]:
def convert_ids_to_text(ids, vocab, eos_idx, unk_idx):
    """ Converts token ids to text.
    """
    if ids.dim() == 1:
        output_tokens = []
        for token_id in ids:
            if token_id == eos_idx:
                break
            else:
                output_tokens.append(vocab.itos[token_id])
        return output_tokens
    
    elif ids.dim() == 2:
        return [convert_ids_to_text(ids[:, i], vocab, eos_idx, unk_idx) for i in range(ids.size(1))]
    
    raise RuntimeError(f'ids has {ids.size()} dimensions, expected 2 dimensions')

In [ ]:
def train_fn(model, iterator, optimizer, criterion, clip=1.0):
    
    
    total_loss = 0
    steps = 0
    model.train()
    tk0 = tqdm(iterator, total=len(iterator), position=0, leave=True)
    
    for idx, batch in enumerate(tk0):
        source = batch.Hindi
        target = batch.roman

        # source: (source_seq_len, batch_size), source_lengths: (batch_size)
        # target: (target_seq_len, batch_size), target_lengths: (batch_size)

        # forward pass
        optimizer.zero_grad()
        output = model(source, target)  # (target_seq_len, batch_size, output_size)
        
        # ignoring SOS_TOKEN
        output = output[1:]  # ((target_seq_len - 1), batch_size, output_size)
        target = target[1:]  # ((target_seq_len - 1), batch_size)
        
        # calculate the loss
        loss = criterion(
            output.view(-1, output.shape[-1]),  # ((target_seq_len - 1) * batch_size, output_size)
            target.view(-1)  # ((target_seq_len - 1) * batch_size)
        )
        total_loss += loss.item()
        steps += 1

        output = output.argmax(dim=-1)  # ((target_seq_len - 1), batch_size)

        # backward pass
        loss.backward()

        # clip gradients to avoid exploding gradients issue
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        # update model parameters
        optimizer.step()
        
        tk0.set_postfix(loss=total_loss/steps)
        
    tk0.close()

    perplexity = np.exp(total_loss / len(iterator))
    
    return output, perplexity
def eval_fn(model, iterator, criterion):
    model.eval()
    total_loss = 0.0
    steps = 0
    hypotheses = []
    references = []
    
    tk0 = tqdm(iterator, total=len(iterator), position=0, leave=True)
    
    with torch.no_grad():
        for idx, batch in enumerate(tk0):
            source = batch.Hindi
            target= batch.roman
            
            # source: (source_seq_len, batch_size), source_lengths: (batch_size)
            # target: (target_seq_len, batch_size), target_lengths: (batch_size)
            
            # forward pass
            output = model(source,target)  # (target_seq_len, batch_size, output_size)
            
            # ignoring SOS_TOKEN
            output = output[1:]  # ((target_seq_len - 1), batch_size, output_size)
            target = target[1:]  # ((target_seq_len - 1), batch_size)
            
            # calculate the loss
            loss = criterion(
                output.view(-1, output.shape[-1]),  # ((target_seq_len - 1) * batch_size, output_size)
                target.view(-1)  # ((target_seq_len - 1) * batch_size)
            )
            total_loss += loss.item()
            steps += 1
            
            output = output.argmax(dim=-1)  # ((target_seq_len - 1), batch_size)

            # converting the ids to tokens (used later for calculating BLEU score)
            pred_tokens = convert_ids_to_text(output, en_text.vocab, EOS_IDX, UNK_IDX)
            target_tokens = convert_ids_to_text(target, en_text.vocab, EOS_IDX, UNK_IDX)

            hypotheses += pred_tokens
            references += [[token] for token in target_tokens]
            
            tk0.set_postfix(loss=total_loss/steps)
            
    tk0.close()
            
    perplexity = np.exp(total_loss / len(iterator))
    bleu4 = bleu_score(hypotheses, references)

    return output, perplexity, bleu4

In [ ]:
cls = Encoder(input_size=INPUT_SIZE, embed_size=ENC_EMBED_SIZE, hidden_size =HIDDEN_SIZE, n_layers=N_LAYERS, dropout_rate=DROPOUT_RATE )
dls = DecoderWithAttention(output_size =OUTPUT_SIZE , embed_size = DEC_EMBED_SIZE, hidden_size=HIDDEN_SIZE, n_layers=N_LAYERS, dropout_rate=DROPOUT_RATE )
model = Seq2SeqWithAttention(cls,dls,pad_idx=PAD_IDX)
if device == 'cuda':
  model.cuda()

In [ ]:
for i in train_dataloader:
  x = cls(i.Hindi,i.roman)
  break

In [ ]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=3e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=3, threshold=0.001, mode='max'
)


In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


In [ ]:
best_bleu4 = float('-inf')
es_patience = 10

patience = 0
model_path = 'model(rms).pth'


for epoch in range(0, N_EPOCHS + 1):
    # one epoch training
    _, train_perplexity = train_fn(model, train_dataloader, optimizer, criterion, CLIP)
    
    # one epoch validation
    _, valid_perplexity, valid_bleu4 = eval_fn(model, val_dataloader, criterion)
    
    print(f'Epoch: {epoch}, Train perplexity: {train_perplexity:.4f}, Valid perplexity: {valid_perplexity:.4f}, Valid BLEU4: {valid_bleu4:.4f}')
    
    scheduler.step(valid_bleu4)
    
    # early stopping
    is_best = valid_bleu4 > best_bleu4
    if is_best:
        print(f'BLEU score improved ({best_bleu4:.4f} -> {valid_bleu4:.4f}). Saving Model!')
        best_bleu4 = valid_bleu4
        patience = 0
        torch.save(model.state_dict(), model_path)
    else:
        patience += 1
        print(f'Early stopping counter: {patience} out of {es_patience}')
        if patience == es_patience:
            print(f'Early stopping! Best BLEU4: {best_bleu4:.4f}')
            break


  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 0, Train perplexity: 10606674071876.3184, Valid perplexity: 86572276389.1072, Valid BLEU4: 0.0000
BLEU score improved (-inf -> 0.0000). Saving Model!


  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 1, Train perplexity: 11465944001486.9590, Valid perplexity: 81349205712.7947, Valid BLEU4: 0.0000
Early stopping counter: 1 out of 10


  0%|          | 0/138 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
model.eval()
for i in val_dataloader:
  input = i.Hindi
  target = i.roman
  output = model(input,target)
  output = output[1:]  # ((target_seq_len - 1), batch_size, output_size)
  target = target[1:]
  print(output.shape)
  output = output.argmax(dim=-1) 
  pred_words = convert_ids_to_text(output, en_text.vocab, EOS_IDX, UNK_IDX)
  actual = convert_ids_to_text(target, en_text.vocab, EOS_IDX, UNK_IDX)
  print(pred_words)
  
  break
  

torch.Size([49, 32, 21955])
[['aisi', 'ek', 'Sachivalay', 'ko', 'paryay', 'madhyam', 'paryay', 'madhyam', 'paryay', 'madhyam', 'paryay', 'madhyam', 'prakar', 'prakar', 'ek', 'Sachivalay', 'utarti', 'aarambh', 'Sachivalay', 'prakar', 'ek', 'Sachivalay', 'prakar', 'ek', 'prakar', 'prakar', 'ek', 'Sachivalay', 'ho', 'paryay', 'Sachivalay', 'prakar', 'utarti', 'prakar', 'prakar', 'ek', 'Sachivalay', 'prakar', 'prakar', 'prakar', 'ek', 'Sachivalay', 'prakar', 'ek', 'Sachivalay', 'ho', 'prakar', 'ek', 'Sachivalay'], ['Sachivalay', 'sanvad', 'Peshewar', 'Sachivalay', 'bheji', 'usa', 'ki.', '(v', 'naam', 'Sachivalay', 'prakash', 'liye', 'Sachivalay', 'prakash', 'liye', 'Sachivalay', 'Mishra', 'paryay', 'Sachivalay', 'tara', 'tara', 'paryay', 'Sachivalay', 'paryay', 'Sachivalay', 'Sachivalay', 'Peshewar', 'Sachivalay', 'paryay', 'paryay', 'paryay', 'paryay', 'paryay', 'Peshewar', 'bheji', 'paryay', 'paryay', 'bodo,', 'Peshewar', 'bheji', 'paryay', 'paryay', 'Peshewar', 'Sachivalay', 'paryay', '

In [ ]:
native_uni = []
native_bi = []
for line in native_corpus:
  #words = words.split(chr(2404))
  #words[0] += " "+chr(2404)
  #words = words[0].split(" ")
  words = line.split(" ")
  native_uni.append(words)
  #bi_words = ["".join(words[i:i+2]) for i in range(0,len(words)-1,1)]


NameError: ignored

In [ ]:
del native_corpus
del df2

In [ ]:
native_uni

In [ ]:
import gensim
from gensim.models import FastText
from gensim.models.word2vec import Word2Vec
# training Word2Vec
# Skip gram word to vec 
# uni grams , bigrams, 


cores = multiprocessing.cpu_count()
cores


2

In [ ]:

fast_t = FastText(native_uni, size=150, window=5, min_count=4, workers=cores,sg=1)


In [ ]:
w2v = gensim.models.word2vec.Word2Vec(native_uni, size=150,   
            window= 8, min_count=5, sg=1, iter=15,workers=cores)

In [ ]:
w2v.save(pth_w2v_r_2)

In [ ]:
print(w2v.similar_by_word("age"))

[('Halanki,', 0.9745258092880249), ('aadhaar', 0.9696657061576843), ('prabhu', 0.9691759347915649), ('laga.', 0.9690280556678772), ('dhan', 0.9684903025627136), ('halanki,', 0.9662507772445679), ('sambhavana', 0.9659699201583862), ('sagar', 0.9651817679405212), ('pushti', 0.9646499156951904), ('paise', 0.9644907116889954)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
!pip install upgrade gensim 


ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [ ]:

w2v = Word2Vec.load(pth_w2v_1)
#w2v_r = Word2Vec.load(pth_w2v_r_2)

In [ ]:
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

In [ ]:
tokenizer_2 = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="<UNK>", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer_2.fit_on_texts(native_corpus)
dic_vocabulary = tokenizer_2.word_index
## create sequence
lst_text2seq= tokenizer_2.texts_to_sequences(native_corpus)
## padding sequence
#X_train = kprocessing.sequence.pad_sequences(lst_text2seq, 
 #                   maxlen=50, padding="post", truncating="post")
Y_train = kprocessing.sequence.pad_sequences(lst_text2seq, 
                   maxlen=50, padding="post", truncating="post")

In [ ]:
Y_train

array([[   51,   602,     6, ...,     0,     0,     0],
       [ 6279,     4,  3827, ...,     0,     0,     0],
       [ 1260,  6281,    23, ...,     0,     0,     0],
       ...,
       [   67, 18547,     6, ...,     0,     0,     0],
       [  259, 18551,  1411, ...,     0,     0,     0],
       [18554,    40,     2, ...,     0,     0,     0]], dtype=int32)

In [ ]:
import pickle

In [ ]:
embedd_native = np.zeros((len(dic_vocabulary)+1, 150))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embedd_native[idx] =  w2v[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
#Dump save embeddings roman and native
with open(path_embedw2v_150_roman, 'wb') as handle:
  pickle.dump(embedd_native, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#load embeddings
with open(path_embedw2v_150_native, 'rb') as handle:
  embeddings = pickle.load(handle)

In [ ]:
# jsonify tokenizer and save with io buffer
jso = tokenizer_2.to_json()
with io.open(path_tokenizer_roman,'w', encoding='utf-8')  as f:
  f.write(json.dumps(jso, ensure_ascii=False))

In [ ]:
# load tokenizer from keras 
from keras_preprocessing.text import tokenizer_from_json
with open(tokenizer_path) as f:
  data = json.load(f)
  tokenizer = tokenizer_from_json(data)

In [ ]:
#save X_train Y train
with open(path_ytrain_roman, 'wb') as handle:
  pickle.dump(Y_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
w2v.predict_output_word()